In [1]:
from tensorflow import keras as tensorflow_keras
import json

2024-10-11 00:49:25.226534: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-11 00:49:25.249083: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 00:49:25.249100: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 00:49:25.249767: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-11 00:49:25.253961: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def get_deep_model():
	model = tensorflow_keras.models.Sequential()
	model.add(tensorflow_keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
	model.add(tensorflow_keras.layers.Conv2D(32, (3, 3), activation='relu'))
	model.add(tensorflow_keras.layers.MaxPooling2D(pool_size=(2, 2)))
	model.add(tensorflow_keras.layers.Dropout(0.25))

	model.add(tensorflow_keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
	model.add(tensorflow_keras.layers.Conv2D(64, (3, 3), activation='relu'))
	model.add(tensorflow_keras.layers.MaxPooling2D(pool_size=(2, 2)))
	model.add(tensorflow_keras.layers.Dropout(0.25))

	model.add(tensorflow_keras.layers.Flatten())
	model.add(tensorflow_keras.layers.Dense(512, activation='relu'))
	model.add(tensorflow_keras.layers.Dropout(0.5))
	model.add(tensorflow_keras.layers.Dense(10, activation='softmax'))
	return model

In [3]:
def get_shallow_model():
	model = tensorflow_keras.models.Sequential()
	model.add(tensorflow_keras.layers.Conv2D(16, (5, 5), padding='same', activation='relu', input_shape=(32, 32, 3)))
	model.add(tensorflow_keras.layers.Flatten())
	model.add(tensorflow_keras.layers.Dense(10, activation='softmax'))
	return model

In [2]:
def get_pretrained_model():
	base_model = tensorflow_keras.applications.MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights='imagenet')
	# Add custom layers on top of MobileNetV2
	model = tensorflow_keras.models.Sequential()
	model.add(base_model)
	model.add(tensorflow_keras.layers.GlobalAveragePooling2D())
	model.add(tensorflow_keras.layers.Dense(512, activation='relu'))
	model.add(tensorflow_keras.layers.Dense(100, activation='softmax'))  # 100 classes for CIFAR-100
	# Freeze first layers until block_16_expand
	for layer in model.layers[0].layers:
		layer.trainable = False
	# Unfreeze the rest
	for layer in model.layers[0].layers[9*16:]:
		layer.trainable = True
	return model

In [3]:
def save_weights(model, path):
    weights = model.get_weights()
    weights_list = [weights_elem.tolist() for weights_elem in weights]

    with open(path, "w") as file:
        json.dump(weights_list, file)

In [4]:
def save_model(model, path):
	model_architecture_str = model.to_json()

	with open(path, "w") as file:
		file.write(model_architecture_str)

# Pretrained model (thought for CIFAR100 dataset)

In [5]:
selected_model = get_pretrained_model()
save_model(selected_model, "./models/pretrained_model")
save_weights(selected_model, "./models/pretrained_model_weights")
print("pretrained_model saved")


2024-10-11 00:49:42.030672: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-11 00:49:42.060517: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-11 00:49:42.060559: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-11 00:49:42.063056: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-11 00:49:42.063101: I external/local_xla/xla/stream_executor

pretrained_model saved


# Deep model (thought for CIFAR10 dataset)

In [8]:
selected_model = get_deep_model()
save_model(selected_model, "./models/deep_model")
save_weights(selected_model, "./models/deep_model_weights")
print("deep_model saved")

deep_model saved


# Shallow model (thought for the MNIST dataset) 

In [9]:
selected_model = get_shallow_model()
save_model(selected_model, "./models/shallow_model")
save_weights(selected_model, "./models/shallow_model_weights")
print("shallow_model saved")

shallow_model saved
